# Image classification 

Used Tensorflow.keras

In [8]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

### Dataset preprocessing 

In [9]:
dataset_path = "data_set"
img_size = (224, 224)
batch_size = 8

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=5,
    zoom_range=0.1,
    brightness_range=[0.9, 1.1]
)

train_data = datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_data = datagen.flow_from_directory(
    dataset_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 75 images belonging to 8 classes.
Found 16 images belonging to 8 classes.


### Model building

In [10]:
base_model = MobileNetV2(include_top=False, input_shape=(224, 224, 3), weights='imagenet')
base_model.trainable = False  

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(train_data.num_classes, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


### Model training

In [11]:
model.fit(train_data, validation_data=val_data, epochs=10)


Epoch 1/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 382ms/step - accuracy: 0.2939 - loss: 1.8471 - val_accuracy: 0.6250 - val_loss: 1.1908
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 237ms/step - accuracy: 0.7554 - loss: 0.7666 - val_accuracy: 0.7500 - val_loss: 0.7242
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 250ms/step - accuracy: 0.8943 - loss: 0.4570 - val_accuracy: 0.8750 - val_loss: 0.5341
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 239ms/step - accuracy: 0.9744 - loss: 0.1589 - val_accuracy: 0.8125 - val_loss: 0.5407
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 231ms/step - accuracy: 1.0000 - loss: 0.0925 - val_accuracy: 0.8750 - val_loss: 0.3952
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 234ms/step - accuracy: 1.0000 - loss: 0.0663 - val_accuracy: 0.9375 - val_loss: 0.3774
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 231ms/step - accuracy: 1.0000 - loss: 0.0605 - val_accuracy: 0.8750 - val_loss: 0.4002
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 235ms/step - accuracy: 1.0000 - loss: 0.0366 - val_accuracy: 0.

In [12]:
model.save("youtube_page_classifier.h5")

### Prediction fuction

In [13]:
def predict_image(image_path):
    img = load_img(image_path, target_size=img_size)
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    prediction = model.predict(img_array)
    class_idx = np.argmax(prediction)
    confidence = np.max(prediction)
    class_labels = list(train_data.class_indices.keys())
    print(f"Predicted Page Type: {class_labels[class_idx]} (Confidence: {confidence:.2f})")
    return class_labels[class_idx]

### Result

In [16]:
predict_image("test11.png")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Predicted Page Type: 0033_shorts (Confidence: 0.99)


'0033_shorts'